# Calculating the reserve Factors of the panels 

# Please enter your name 

In [10]:
name = "yannis"

# Imports 

In [11]:
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))
sys.path.insert(0, os.path.abspath('../optimization'))

from formulas.panels import *
from formulas.helpers import *
from formulas.abd_matrix import *

 

In [12]:
'''# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])'''

"# config parser\nimport configparser\n\n# get the rounding_digits from the ini file\nconfig = configparser.ConfigParser()\nconfig.read('../config.ini')\nrounding_digits = int(config['DEFAULT']['rounding_digits'])"

# Constants (Changed based on your data)

In [13]:
personal_data = personal_data_provider(name)
E_11 = personal_data[0]
E_22 = personal_data[1]
G_12 = personal_data[2]
nu_12 = personal_data[3]
print(f"Your personal data is: E11 = {E_11}, E22 = {E_22}, G12 = {G_12}, nu12 = {nu_12}. Please verify.")

# Panel laminate properties 
panelStack = [45,45,-45,-45,0,0,90,90,90,90,0,0,-45,-45,45,45]
tPanel = 0.552 

#Panel constants 
length = 750
width = 400


Your personal data is: E11 = 119325.52800000002, E22 = 9178.884, G12 = 4589.442, nu12 = 0.33. Please verify.


# Import needed ABD values 

In [14]:
ABD_panel, ABD_panel_inverse = calculateABD(stacksequence=panelStack, plyT=tPanel, EModulus1=E_11, EModulus2=E_22, ShearModulus=G_12)

# Open stress values & properties of panels 

In [15]:
# New version of importing panel stress file
paneldf = pd.read_csv(f'../data/{name}/panel.csv')
paneldf = paneldf.rename(columns={'Elements':'Element ID', 'XX':'sigmaXX', 'YY':'sigmaYY', 'XY':'sigmaXY', 'Loadcase':'Load Case'})
paneldf['Component Name'] = paneldf.apply(elementComponentMatch, axis=1)
paneldf = paneldf.drop(['FileID', 'Step', 'Layer'], axis=1)

paneldf.head(5)


,Element ID,Load Case,sigmaXX,sigmaXY,sigmaYY,Component Name
0,1,1,-41.655796,16.694225,5.143600,panel1
1,2,1,-50.504929,13.890759,6.055713,panel1
2,3,1,-41.692043,14.728354,13.374496,panel1
3,4,1,-41.826157,15.569081,7.285109,panel1
4,5,1,-38.997730,15.624921,6.635034,panel1


# Split up the loadcases 

In [16]:
loadCase1df = paneldf[paneldf['Load Case'] == 1]
loadCase2df = paneldf[paneldf['Load Case'] == 2]
loadCase3df = paneldf[paneldf['Load Case'] == 3]


#Set index back to Element ID 
loadCase1df = loadCase1df.set_index('Element ID')
loadCase2df = loadCase2df.set_index('Element ID')
loadCase3df = loadCase3df.set_index('Element ID')

#Give some overview 
loadCase1df.head(5)

,Load Case,sigmaXX,sigmaXY,sigmaYY,Component Name
Element ID,,,,,
1,1,-41.655796,16.694225,5.143600,panel1
2,1,-50.504929,13.890759,6.055713,panel1
3,1,-41.692043,14.728354,13.374496,panel1
4,1,-41.826157,15.569081,7.285109,panel1
5,1,-38.997730,15.624921,6.635034,panel1


# Now we need to combine into panels:
-Panel ID (as our ID)
-sigmaXXavg
-sigmaYYavg
-sigmaXYavg
-length
-width
-thickness 


In [17]:
# We can just average the stresses @fabianober Tackle rounding!!!! 
#For load case 1 
panelLC1 = loadCase1df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
})
dataframeLength = len(panelLC1)
panelLC1['length'] = [length]*dataframeLength
panelLC1['width'] = [width]*dataframeLength
panelLC1['Load Case'] = ['LC1'] * dataframeLength


#For load case 2 
panelLC2 = loadCase2df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean'
})
dataframeLength = len(panelLC2)
panelLC2['length'] = [length]*dataframeLength
panelLC2['width'] = [width]*dataframeLength
panelLC2['Load Case'] = ['LC2'] * dataframeLength


#For load case 3
panelLC3 = loadCase3df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean'
})
dataframeLength = len(panelLC3)
panelLC3['length'] = [length]*dataframeLength
panelLC3['width'] = [width]*dataframeLength
panelLC3['Load Case'] = ['LC3'] * dataframeLength


panelLC1

,sigmaXX,sigmaYY,sigmaXY,length,width,Load Case
Component Name,,,,,,
panel1,-42.697845,8.136578,15.483408,750,400,LC1
panel2,-39.443552,8.188004,16.935501,750,400,LC1
panel3,-39.528903,8.051509,17.415443,750,400,LC1
panel4,-39.395589,8.160427,16.368667,750,400,LC1
panel5,-42.484748,8.082001,14.220927,750,400,LC1


# Now we can apply the functions to the load cases 

In [ ]:
panelLC1[['sig_crit_shear', 'sig_crit_biax', 'Reserve Factor']] = panelLC1.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC2[['sig_crit_shear', 'sig_crit_biax', 'Reserve Factor']] = panelLC2.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC3[['sig_crit_shear', 'sig_crit_biax', 'Reserve Factor']] = panelLC3.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC1

NameError: name 'EModulus' is not defined

# Create original output 

In [ ]:
#Drop irrelevant columns
panelLC1 = panelLC1.drop(['length', 'width', 'Load Case'], axis=1)
panelLC2 = panelLC2.drop(['length', 'width',  'Load Case'], axis=1)
panelLC3 = panelLC3.drop(['length', 'width',  'Load Case'], axis=1)
panelLC1.head(3)

,sigmaXX,sigmaYY,sigmaXY,k_shear,k_biaxial,Reserve Factor
Component Name,,,,,,
panel1,-73.707667,17.880333,29.722000,5.624444,5.026109,1.582573
panel10,-73.707667,17.880333,29.722000,5.624444,5.026109,1.582573
panel2,-93.826667,17.879667,29.643333,5.624444,4.824758,1.330840


In [ ]:
# Rename columns to be unambigous
panelLC1 = panelLC1.rename(columns={"Reserve Factor": "LC 1 RF", 'sigmaXX':'sigmaXXLC1', 'sigmaYY':'sigmaYYLC1', 'sigmaXY':'sigmaXYLC1', 'sig_crit_shear':'sig_crit_shearLC1', 'sig_crit_biax':'sig_crit_biaxLC1'})
panelLC2 = panelLC2.rename(columns={"Reserve Factor": "LC 2 RF", 'sigmaXX':'sigmaXXLC2', 'sigmaYY':'sigmaYYLC2', 'sigmaXY':'sigmaXYLC2', 'sig_crit_shear':'sig_crit_shearLC2', 'sig_crit_biax':'sig_crit_biaxLC2'})
panelLC3 = panelLC3.rename(columns={"Reserve Factor": "LC 3 RF", 'sigmaXX':'sigmaXXLC3', 'sigmaYY':'sigmaYYLC3', 'sigmaXY':'sigmaXYLC3', 'sig_crit_shear':'sig_crit_shearLC3', 'sig_crit_biax':'sig_crit_biaxLC3'})
panelLC1

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,k_shearLC1,k_biaxialLC1,LC 1 RF
Component Name,,,,,,
panel1,-73.707667,17.880333,29.722000,5.624444,5.026109,1.582573
panel10,-73.707667,17.880333,29.722000,5.624444,5.026109,1.582573
panel2,-93.826667,17.879667,29.643333,5.624444,4.824758,1.330840
panel3,-94.244667,17.880000,29.593333,5.624444,4.820475,1.214163
panel4,-92.911667,17.880000,30.065000,5.624444,4.834354,1.489082
panel5,-92.166667,17.880000,52.546333,5.624444,4.842322,2.061408
panel6,-92.166667,17.880000,52.546333,5.624444,4.842322,2.061408
panel7,-92.911667,17.880000,30.065000,5.624444,4.834354,1.489082
panel8,-94.244667,17.880000,29.593333,5.624444,4.820475,1.214163


In [ ]:
outputDf = pd.concat([panelLC1, panelLC2, panelLC3], axis=1)
outputDf

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,k_shearLC1,k_biaxialLC1,LC 1 RF,sigmaXXLC2,sigmaYYLC2,sigmaXYLC2,k_shearLC2,k_biaxialLC2,LC 2 RF,sigmaXXLC3,sigmaYYLC3,sigmaXYLC3,k_shearLC3,k_biaxialLC3,LC 3 RF
Component Name,,,,,,,,,,,,,,,,,,
panel1,-73.707667,17.880333,29.722000,5.624444,5.026109,1.582573,-76.008667,-17.880000,29.722000,5.624444,3.072999,0.966167,-24.486667,5.94,118.488667,5.624444,5.026098,1.103253
panel10,-73.707667,17.880333,29.722000,5.624444,5.026109,1.582573,-76.008667,-17.880000,29.722000,5.624444,3.072999,0.966167,-24.486667,5.94,118.488667,5.624444,5.026098,1.103253
panel2,-93.826667,17.879667,29.643333,5.624444,4.824758,1.330840,-93.875000,-17.880000,29.643333,5.624444,3.238663,0.907764,-31.170667,5.94,118.176000,5.624444,4.824774,1.201583
panel3,-94.244667,17.880000,29.593333,5.624444,4.820475,1.214163,-94.625667,-17.880000,29.593333,5.624444,3.244567,0.828535,-31.309667,5.94,117.976000,5.624444,4.820469,1.043226
panel4,-92.911667,17.880000,30.065000,5.624444,4.834354,1.489082,-93.657667,-17.879667,30.065000,5.624444,3.236955,1.005024,-30.866667,5.94,119.855000,5.624444,4.834353,1.396748
panel5,-92.166667,17.880000,52.546333,5.624444,4.842322,2.061408,-93.152333,-17.880000,52.546333,5.624444,3.232911,1.407375,-30.619333,5.94,209.480667,5.624444,4.842316,1.132782
panel6,-92.166667,17.880000,52.546333,5.624444,4.842322,2.061408,-93.152333,-17.880000,52.546333,5.624444,3.232911,1.407375,-30.619333,5.94,209.480667,5.624444,4.842316,1.132782
panel7,-92.911667,17.880000,30.065000,5.624444,4.834354,1.489082,-93.657667,-17.879667,30.065000,5.624444,3.236955,1.005024,-30.866667,5.94,119.855000,5.624444,4.834353,1.396748
panel8,-94.244667,17.880000,29.593333,5.624444,4.820475,1.214163,-94.625667,-17.880000,29.593333,5.624444,3.244567,0.828535,-31.309667,5.94,117.976000,5.624444,4.820469,1.043226


# ROUND & Output the files

In [ ]:
outputDf = outputDf.round(rounding_digits)
outputDf.to_csv(f'../data/{name}/output/processed_e.csv')
outputDf.head(10)
updateParametersDf = updateParametersDf.round(rounding_digits)
updateParametersDf.to_csv(f'../data/{name}/output/updatePanelThick.csv')
updateParametersDf.head(10)

,thickness
Component Name,
panel1,3.83
panel10,3.83
panel2,4.15
panel3,4.16
panel4,4.14
panel5,4.70
panel6,4.70
panel7,4.14
panel8,4.16
